In [1]:
import os
import sys
import csv
import pandas as pd
import matplotlib as plt
%matplotlib inline
import numpy as np
from sklearn.preprocessing import LabelEncoder
import seaborn as sns


In [2]:
from keras.layers import Reshape, Embedding, Conv1D, MaxPooling1D,GlobalMaxPooling1D, Flatten, Dense,LSTM, GRU,Bidirectional,Dropout, Conv3D
from tensorflow.keras import Input
from keras.initializers import Constant
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical
from tensorflow.keras import layers
from tensorflow import keras
import keras.backend as K
from keras.layers import BatchNormalization
import tensorflow as tf
from keras.optimizers import RMSprop
from keras.callbacks import ModelCheckpoint
from tensorflow.keras.callbacks import EarlyStopping
from tqdm import tqdm



In [3]:
from gensim.models import Word2Vec, KeyedVectors
from collections import defaultdict

In [4]:
## for bag-of-words
from sklearn import feature_extraction, model_selection, manifold, preprocessing, feature_selection
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import SGDClassifier
from sklearn.pipeline import Pipeline

 ## Data Acquisition




In [5]:
def downloadDataset():
  if  not os.path.isfile('./drive/MyDrive/QS-OCR-small.tar.gz'):
    print("Downloading... ")
    ! wget https://github.com/QuickSign/ocrized-text-dataset/releases/download/v1.0/QS-OCR-small.tar.gz -P ./drive/MyDrive/
    !mkdir ./drive/MyDrive/dataset
    !tar -xf QS-OCR-small.tar.gz -C ./drive/MyDrive/dataset
  else: 
    print("Dataset has already been downloaded. ")

In [6]:
def createSingleDataset(filename):
  # join all the file content into a single csv file
  if not os.path.isfile(filename):
    print("joining..")
    path = "./drive/MyDrive/dataset"
    file_dataset = open(filename, mode='w')
    file_writer = csv.writer(file_dataset, delimiter='|', quotechar='"', quoting=csv.QUOTE_MINIMAL)
    #walk trough the directory to get all the files' content
    for dirpath, dirs, filenames in os.walk(path):
      for filename in filenames:
        f = open(dirpath +"/"+filename, "r")
        getfilebody = f.read()
        file_writer.writerow([filename, getfilebody,  dirpath.replace("./drive/MyDrive/dataset/", "") ] )
        f.close()
  else:
      print("Dataset files have already been joined. ")

In [7]:
def get_data(): 
  downloadDataset()
  createSingleDataset('./drive/MyDrive/dataset.csv')
  df = pd.read_csv("./drive/MyDrive/dataset.csv", sep='|', delimiter=None, names = ["filename","content", "label"] )
  return df

In [8]:
import tensorflow as tf
tf.test.gpu_device_name()

''

##Pre-Processing
Data cleaning:
  >* filling in missing values,
  >*  smoothing noisy data,
  >* identifying or removing outliers,
  >* resolving inconsistencies,
  >* duplicate samples
  >* NLP pipeline = data normalizzation for text
  >>* Tokenization
  >>* punctuation removal
  >>* stop words removal
  >>* noise removal: Remove HTML tags, remove extra whitespaces, 
  >>* normalization: typo correction,  converting all characters to lowercase, number, date, symbol textualization, Expand contractions, remove_accented_chars
  >>* lemmatization o steaming 


In [9]:
def remove_tags(string):
    result = re.sub('<.*?>',' ',string)
    result = re.sub(r'\b\w{1,3}\b', '', result) #remove words with legth < 3 chars
    return result

def clean_text(text):
  for t in nlp.tokenizer(text):
    if ((not t.is_stop) and ( not t.is_punct) and (not t.is_currency) and (not t.is_digit) and (not t.is_space) and (t.is_alpha) ):
      return t.lemma_.lower().join(t.pos_)

def execute_nlp(df):
  nlp = spacy.load('en')  
  #speech tag removed because it's too slow
  df.insert(1, 'cleaned_text', df['content_without_tags'].apply(lambda x: [t.lemma_.lower() for t in nlp.tokenizer(x) if ((not t.is_stop) and ( not t.is_punct) and (not t.is_currency) and (not t.is_digit) and (not t.is_space) and (t.is_alpha) ) ]))
  return df



In [10]:

!pip install langid
import spacy 
import en_core_web_sm
import langid
import re as re

def pre_processing(df):
  # nan removing
  df = df.dropna()

  #label encoding
  categories = pd.unique(df["label"])
  labelencoder = LabelEncoder()
  df['y'] = labelencoder.fit_transform(df['label'])
  
  #nlp pipeline

  df['content']=df['content'].apply(str)
  df.insert(1, 'content_without_tags', df["content"].apply(lambda cw : remove_tags(cw)))  #removing HTML tags
  df = execute_nlp(df)        

  #remove unuseful column
  df = df.drop("content_without_tags", axis = 1)
  df = df.drop("filename", axis = 1)
  
  return df , categories


## Visualization and Dataset Analysis

In [11]:
!pip install wordcloud
!pip install pillow
from wordcloud import WordCloud, ImageColorGenerator
from PIL import Image

def word_cloud(df):
   
  comment_words = '' 
  
  # iterate through the csv file 
  for val in df.cleaned_text: 
        
      # typecaste each val to string 
      val = str(val) 
    
      # split the value 
      tokens = val.split() 
        
      # Converts each token into lowercase 
      for i in range(len(tokens)): 
          tokens[i] = tokens[i].lower() 
        
      comment_words += " ".join(tokens)+" "
    
  wordcloud = WordCloud(width = 800, height = 800, 
                  background_color ='white', 
                  min_font_size = 10).generate(comment_words) 
      
    # plot the WordCloud image                        
  plt.pyplot.figure(figsize = (8, 8), facecolor = None) 
  plt.pyplot.imshow(wordcloud) 
  plt.pyplot.axis("off") 
  plt.pyplot.title( pd.unique(df.label))

  plt.pyplot.tight_layout(pad = 0) 
    
  plt.pyplot.show() 



### Utils

In [12]:
## split dataset
def split_dataset(X, y):
  #X = df.iloc[:, 0 ]
  # y = df.iloc[:, 4 ]
  X_train, X_test, y_train, y_test= model_selection.train_test_split(X, y, test_size=0.3, random_state = 42)
  X_train, X_val, y_train, y_val = model_selection.train_test_split(X_train, y_train, test_size=0.2, random_state = 42)

  #y_test = y_test.values.reshape((1045,1))
  return X_train, X_test, y_train, y_test, X_val, y_val



In [13]:
def get_most_frequent_word(number_of_word_to_get):
  sentences = [row.split() for row in df['cleaned_text']] #tokenize sentence

  #create dictionary word_freq(word, number_of_occurence)
  word_freq = defaultdict(int)
  for sent in sentences:
      for i in sent:
        # print(sent)
          #print(i)
          word_freq[i] += 1
          
  print("top 10 most frequent words:", sorted(word_freq, key=word_freq.get, reverse=True)[:number_of_word_to_get])
  #word_freq



In [14]:
def get_f1(y_true, y_pred): #taken from old keras source code
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    recall = true_positives / (possible_positives + K.epsilon())
    f1_val = 2*(precision*recall)/(precision+recall+K.epsilon())
    
    return f1_val

In [15]:
def getPerformanceMetricsBERT(model, X_test, y_test, modelName):
  print(modelName, "metrics: ")
  from sklearn.metrics import classification_report, accuracy_score,confusion_matrix,roc_curve,auc,precision_recall_curve
  y_pred = model.predict(X_test)
  #predicted_prob = model.predict_proba(X_test)
  print('accuracy %s' % accuracy_score(y_pred.round(), y_test))
  print(classification_report(y_test, y_pred.round(), target_names=categories))
   ## Plot confusion matrix
  #print("confusion Matrix", modelName)
  cm = confusion_matrix(y_test.argmax(axis=1), y_pred.argmax(axis=1))
  fig, ax = plt.pyplot.subplots()
  sns.heatmap(cm, annot=True, fmt='d', ax=ax, cmap=plt.cm.Blues, 
              cbar=False)
  ax.set(xlabel="Pred", ylabel="True", xticklabels=categories, 
        yticklabels=categories, title="Confusion matrix "+modelName)
  plt.pyplot.yticks(rotation=0)
  plt.pyplot.show()

In [16]:
def plot_accuracy_loss(history): 
  plt.pyplot.subplot(211)
  plt.pyplot.plot(history.history['get_f1'])
  plt.pyplot.title('model f1_score')
  plt.pyplot.ylabel('f1_score')
  plt.pyplot.plot(history.history['val_get_f1'])
  plt.pyplot.xlabel('epoch')
  plt.pyplot.legend(['train', 'val'], loc='upper left')
  plt.pyplot.subplot(212)
  plt.pyplot.plot(history.history['loss'])
  plt.pyplot.title('val loss')
  plt.pyplot.ylabel('loss')
  plt.pyplot.plot(history.history['val_loss'])
  plt.pyplot.xlabel('epoch')
  plt.pyplot.legend(['train', 'val'], loc='upper left')
  plt.pyplot.show()

## Baseline Model 

In [17]:

def multinomialNBmodel():
  nb = Pipeline([('vectorizer', feature_extraction.text.CountVectorizer()),
                ('tfidf', feature_extraction.text.TfidfTransformer()),
               ('clf', MultinomialNB()),])
  return nb
def SVMmodel():
  svm = Pipeline([('vectorizer', feature_extraction.text.CountVectorizer()),
                ('tfidf', feature_extraction.text.TfidfTransformer()),
               ('clf',  SGDClassifier(penalty='l2', alpha=1e-3, random_state=42)),])
  return svm

def getPerformanceMetrics(model, X_test, y_test):
  from sklearn.metrics import classification_report, accuracy_score,confusion_matrix,roc_curve,auc,precision_recall_curve
  y_pred = model.predict(X_test)
  #predicted_prob = model.predict_proba(X_test)
  print('accuracy %s' % accuracy_score(y_pred, y_test))
  print(classification_report(y_test, y_pred, target_names=categories))
   ## Plot confusion matrix
  cm = confusion_matrix(y_test, y_pred)
  fig, ax = plt.pyplot.subplots()
  sns.heatmap(cm, annot=True, fmt='d', ax=ax, cmap=plt.cm.Blues, 
              cbar=False)
  ax.set(xlabel="Pred", ylabel="True", xticklabels=categories, 
        yticklabels=categories, title="Confusion matrix")
  plt.pyplot.yticks(rotation=0)
#  fig, ax = plt.pyplot.subplots(nrows=1, ncols=2)
  

#NOTE: with MultinomialNB and tfidf there are label that are not predicted 

In [18]:
def compute_baseline(df):#model = multinomialNBmodel()
  X_train, X_test, y_train, y_test, X_val, y_val = split_dataset(df.iloc[:, 0 ],df.iloc[:, 4])  #cleaned text and string label
  model = SVMmodel()
  model.fit(X_train, y_train)
  getPerformanceMetrics(model, X_test, y_test)


## Modeling - Word Embeddings

####Create Customized Word Embedding

In [19]:
def create_custom_word_embedding(df):
    sentences = [row.split() for row in df['cleaned_text']] #tokenize sentence
    w2v_model = Word2Vec(min_count=100, #min_count: minimum amount of time a word appear in word2vec training corpus  
                        window=5,  # context window +- center word
                        size=100,  # dimensions of word embeddings, also refer to size of hidden layer
                        workers=4,
                        sg =1 )#1= skip-gram, 0 = cbow
                        
    w2v_model.build_vocab(sentences)
    w2v_model.train(sentences, total_examples=w2v_model.corpus_count, epochs=w2v_model.iter)
    w2v_model.init_sims(replace=True)
    print("vocab length",len(w2v_model.wv.vocab))
    return  w2v_model


In [20]:
#testing word embedding
#Find the most similar words for "smoke"
def test_word_embedding(w2v_model, word):
  print(w2v_model.wv.most_similar(positive=[word]))



In [21]:
from sklearn.manifold import TSNE
def visualize_word2vec_model(model):
    "Create TSNE model and plot it"
    labels = []
    tokens = []

    for word in model.wv.vocab:
        tokens.append(model[word])
        labels.append(word)
    
    tsne_model = TSNE(perplexity=40, n_components=2, init='pca', n_iter=2500, random_state=23)
    new_values = tsne_model.fit_transform(tokens)

    x = []
    y = []
    for value in new_values:
        x.append(value[0])
        y.append(value[1])
        
    plt.pyplot.figure(figsize=(18, 18)) 
    for i in range(len(x)):
        plt.pyplot.scatter(x[i],y[i])
        plt.pyplot.annotate(labels[i],
                     xy=(x[i], y[i]),
                     xytext=(5, 2),
                     textcoords='offset points',
                     ha='right',
                     va='bottom')
    plt.pyplot.show()


In [22]:
def save_pretrained_word_embedding(filename, isBin, w2v_model):
  w2v_model.wv.save_word2vec_format(filename, binary = isBin)


#### Modeling Using Custom Word Embedding

In [23]:
def inits_params_word_embedding(vocab_len, max_sequence_length, val_split, embedding_dim ):
  MAX_NB_WORDS = vocab_len
  MAX_SEQUENCE_LENGTH = max_sequence_length # 800
  VALIDATION_SPLIT = val_split  #0.2
  EMBEDDING_DIM = embedding_dim #300 #has to be between 100 and 300
  
  return MAX_NB_WORDS, MAX_SEQUENCE_LENGTH, VALIDATION_SPLIT, EMBEDDING_DIM


In [24]:
def data_preparation(df,MAX_NB_WORDS,  MAX_SEQUENCE_LENGTH, VALIDATION_SPLIT, EMBEDDING_DIM):
  #convert data into vector to fit word embedding:
    # it means vectorize a text corpus, by turning each text into either a sequence of integers 
  tokenizer = Tokenizer(num_words = MAX_NB_WORDS)
  tokenizer.fit_on_texts(df["cleaned_text"])   #In the case where texts contains lists, we assume each entry of the lists to be a token.
  sequences = tokenizer.texts_to_sequences(df["cleaned_text"])  #turn each list into a "sequence": is a list of integer word indices. 
  word_index = tokenizer.word_index
  print('Found %s unique tokens.' % len(word_index))

  data = pad_sequences(sequences, maxlen=MAX_SEQUENCE_LENGTH) #X

  labels = to_categorical(np.asarray(df["y"])) #y
  print('Shape of data tensor:', data.shape)
  print('Shape of label tensor:', labels.shape)

  # split the data into a training set and a validation set
  
  X_train, X_test, y_train, y_test, x_val, y_val = split_dataset(data, labels)
    
  return word_index, X_train, X_test, y_train, y_test, x_val, y_val, y_test


In [25]:
def load_word_embedding(filename):
  #extracting vocabs from 'word_embedding.txt'
  embeddings_index = {}
  f = open(os.path.join("", filename),encoding='utf-8' )
  for line in tqdm(f):
      values = line.rstrip().rsplit(' ')
      word = values[0]
      coefs = np.asarray(values[1:], dtype='float32')
      embeddings_index[word] = coefs
  f.close()
  print('Found %s word vectors.' % len(embeddings_index))

  return embeddings_index

In [26]:
#create embedding matrix that join pretrained word embedding with the dataset's vocabulary
def create_embedding_matrix(embeddings_index, word_index, EMBEDDING_DIM):
  embedding_matrix = np.zeros((len(word_index) + 1, EMBEDDING_DIM))
  for word, i in word_index.items():
      embedding_vector = embeddings_index.get(word)
      if embedding_vector is not None:
          # words not found in embedding index will be all-zeros.
          embedding_matrix[i] = embedding_vector
  return embedding_matrix


In [27]:
def inits_params_embedding_layer(embedding_matrix, word_index, MAX_SEQUENCE_LENGTH, EMBEDDING_DIM):
  print("embedding_layer")
  embedding_layer = Embedding(len(word_index) + 1,
                            EMBEDDING_DIM,
                            weights=[embedding_matrix],
                            input_length=MAX_SEQUENCE_LENGTH,
                            trainable=False)
  return embedding_layer

In [28]:
def run_model(model,  x_train, y_train,  x_val, y_val, modelName):
  model.compile(loss='categorical_crossentropy',
                optimizer='rmsprop',
                metrics=[get_f1, "accuracy"])
  es_callback = EarlyStopping(monitor='val_loss', patience=3)
  #history = model.fit(x_train, y_train, batch_size=128, epochs=30, validation_data=(x_val, y_val), callbacks=[es_callback], shuffle=False)
  model.summary()
  # happy learning!
  history = model.fit(x_train, y_train, validation_data=(x_val, y_val),
            epochs=20, batch_size=64,  callbacks=[es_callback], shuffle=False)
  model.save(modelName)
  
  plot_accuracy_loss(history)

  print("history", history)
  hist_df = pd.DataFrame(history.history) 

  hist_csv_file = './drive/MyDrive/history/history' + modelName.replace("./drive/MyDrive/", "") + '.csv'   
  with open(hist_csv_file, mode='w') as f:
    hist_df.to_csv(f)
  return model

In [29]:
""" #cose aggiunte
  hist_df = pd.DataFrame(history.history) 

  hist_csv_file = 'history' + modelName.replace("./drive/MyDrive/", "") + '.csv'   
  with open(hist_csv_file, mode='w') as f:
    hist_df.to_csv(f)

  #print("history", history)"""


' #cose aggiunte\n  hist_df = pd.DataFrame(history.history) \n\n  hist_csv_file = \'history\' + modelName.replace("./drive/MyDrive/", "") + \'.csv\'   \n  with open(hist_csv_file, mode=\'w\') as f:\n    hist_df.to_csv(f)\n\n  #print("history", history)'

In [30]:

def define_CNN_model(embedding_layer, MAX_SEQUENCE_LENGTH):

  sequence_input = Input(shape=(MAX_SEQUENCE_LENGTH,), dtype='int32')
  embedded_sequences = embedding_layer(sequence_input)
  x = Conv1D(128, 5, activation='relu')(embedded_sequences)
  x = MaxPooling1D(5)(x)
  x = Conv1D(128, 5, activation='relu')(x)
  x = MaxPooling1D(5)(x)
  x = Conv1D(128, 5, activation='relu')(x)
  x = MaxPooling1D()(x)  # global max pooling
  x = Flatten()(x)
  x = Dense(128, activation='relu')(x)
  preds = Dense(10, activation='softmax')(x)
  
  model = keras.Model(sequence_input, preds)

  return model


In [31]:
def define_LSTM_model(embedding_layer, MAX_SEQUENCE_LENGTH,  word_index): 
  sequence_input = Input(shape=(MAX_SEQUENCE_LENGTH,), dtype='int32')
  modelLSTM = keras.Sequential()
  embedded_sequences = embedding_layer(sequence_input)
  modelLSTM.add(embedding_layer)
  modelLSTM.add(Bidirectional(LSTM(units=32, dropout=0.3,recurrent_dropout=0.2)))
  modelLSTM.add(Dense(32,activation='relu'))
  modelLSTM.add(Dropout(0.3))
  modelLSTM.add(Dense(10,activation="softmax"))
  return modelLSTM


In [32]:
def training_over_custom_word_embedding(df):
  w2v_model = create_custom_word_embedding(df)  
  visualize_word2vec_model(w2v_model)
  save_pretrained_word_embedding("word_embedding.txt", False, w2v_model)

  #get dataset info
  max_words_for_sentence = df["number_of_words"].max()
  median_words_for_sentence = df["number_of_words"].median()
  vocab_len = len(w2v_model.wv.vocab)
  print("vocab length: ",len(w2v_model.wv.vocab), "max", max_words_for_sentence)



  MAX_NB_WORDS, MAX_SEQUENCE_LENGTH, VALIDATION_SPLIT, EMBEDDING_DIM = inits_params_word_embedding(len(w2v_model.wv.vocab), max_words_for_sentence, 0.2, 100 )
  word_index, x_train, x_test, y_train,y_val, x_val, y_val, y_test = data_preparation(df,MAX_NB_WORDS, MAX_SEQUENCE_LENGTH, VALIDATION_SPLIT, EMBEDDING_DIM)

 # X_train, X_test, y_train, y_test, x_val, y_val

  embeddings_index = load_word_embedding('word_embedding.txt')
  embedding_matrix =create_embedding_matrix(embeddings_index, word_index, EMBEDDING_DIM)
  if not os.path.isdir("./drive/MyDrive/CNN_model_custom_WEskipgram") or not os.path.isdir("./drive/MyDrive/LSTM_model_custom_WEskipgram") :
    print("models are training...")
    embedding_layer = inits_params_embedding_layer(embedding_matrix, word_index, MAX_SEQUENCE_LENGTH, EMBEDDING_DIM )
    CNN_model_custom_we = define_CNN_model(embedding_layer, MAX_SEQUENCE_LENGTH) 
    CNN_model_custom_we = run_model(CNN_model_custom_we, x_train, y_train,  x_val, y_val, "./drive/MyDrive/CNN_model_custom_WEskipgram")
    #getPerformanceMetricsBERT(CNN_model_custom_we, x_test, y_test )

    LSTM_model_custom_we = define_LSTM_model(embedding_layer, MAX_SEQUENCE_LENGTH, word_index)
    LSTM_model_custom_we.summary()
    LSTM_model_custom_we = run_model(LSTM_model_custom_we,  x_train, y_train,  x_val, y_val, "./drive/MyDrive/LSTM_model_custom_WEskipgram")

    #getPerformanceMetricsBERT(modelLstm, x_test, y_test )
  else:
    #load trained models
    print("models already trained, loading...")
    LSTM_model_custom_we = tf.keras.models.load_model('./drive/MyDrive/LSTM_model_custom_WEskipgram',  custom_objects={'get_f1': get_f1})
    CNN_model_custom_we = tf.keras.models.load_model('./drive/MyDrive/CNN_model_custom_WEskipgram',  custom_objects={'get_f1': get_f1}) 
  
  return LSTM_model_custom_we, CNN_model_custom_we, x_test, y_test

#### Modeling Using Glove Pre-trained Word Embedding

In [33]:
import requests, zipfile, io
def glove_download(): 
  if  not os.path.isfile('./drive/MyDrive/glove.840B.300d.txt/glove.840B.300d.txt'):
    print("Downloading... ")
    zip_file_url = "http://nlp.stanford.edu/data/glove.840B.300d.zip"
    r = requests.get(zip_file_url)
    z = zipfile.ZipFile(io.BytesIO(r.content))
    z.extractall("./drive/MyDrive/glove.840B.300d.txt")
  else: 
    print("glove has already been downloaded")




In [34]:
def training_over_glove_word_embedding(df):
  glove_download()
  embeddings_index = load_word_embedding('./drive/MyDrive/glove.840B.300d.txt/glove.840B.300d.txt')
  #TODO: define MAX_NB_WORDS correctly
  max_words_for_sentence = df["number_of_words"].max()
  median_words_for_sentence = df["number_of_words"].median()
 # vocab_len = len(w2v_model.wv.vocab)
 # print("vocab length: ",len(w2v_model.wv.vocab), "max", max_words_for_sentence)

  MAX_NB_WORDS, MAX_SEQUENCE_LENGTH, VALIDATION_SPLIT, EMBEDDING_DIM = inits_params_word_embedding(len(embeddings_index), max_words_for_sentence, 0.2, 300 )
  word_index, x_train, x_test, y_train, y_test, x_val, y_val , y_test = data_preparation(df, MAX_NB_WORDS, MAX_SEQUENCE_LENGTH, VALIDATION_SPLIT, EMBEDDING_DIM)

  embedding_matrix =create_embedding_matrix(embeddings_index, word_index, EMBEDDING_DIM)

  
  if not os.path.isdir("./drive/MyDrive/CNN_model_glove_WE") or not os.path.isdir("./drive/MyDrive/LSTM_model_glove_WE") :
    print("models are training...")
    embedding_layer = inits_params_embedding_layer(embedding_matrix, word_index, MAX_SEQUENCE_LENGTH, EMBEDDING_DIM )
    CNN_model_glove_we = define_CNN_model(embedding_layer, MAX_SEQUENCE_LENGTH) 
    CNN_model_glove_we = run_model(CNN_model_glove_we, x_train, y_train,  x_val, y_val, "./drive/MyDrive/CNN_model_glove_WE")
    #getPerformanceMetricsBERT(CNN_model_glove_we, x_test, y_test )

    LSTM_model_glove_we = define_LSTM_model(embedding_layer, MAX_SEQUENCE_LENGTH, word_index)
    LSTM_model_glove_we.summary()
    LSTM_model_glove_we = run_model(LSTM_model_glove_we,  x_train, y_train,  x_val, y_val, "./drive/MyDrive/LSTM_model_glove_WE")
    #getPerformanceMetricsBERT(LSTM_model_glove_we, x_test, y_test )
    
  else:
    #load trained models
    print("models already trained, loading...")
    LSTM_model_glove_we = tf.keras.models.load_model('./drive/MyDrive/LSTM_model_glove_WE',  custom_objects={'get_f1': get_f1})
    CNN_model_glove_we = tf.keras.models.load_model('./drive/MyDrive/CNN_model_glove_WE',  custom_objects={'get_f1': get_f1}) 
  
  return LSTM_model_glove_we, CNN_model_glove_we, x_test, y_test


#### Bert Tensorflow 

In [35]:
!pip install sentencepiece

In [36]:
!wget --quiet https://raw.githubusercontent.com/tensorflow/models/master/official/nlp/bert/tokenization.py

In [37]:
from tensorflow.keras.layers import Dense, Input
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import ModelCheckpoint
import tensorflow_hub as hub

import tokenization

In [38]:
### Add tokens to the data make it BERT compatible
def bert_encode(test_df, tokenizer, max_len=512):
    all_tokens = []
    all_masks = []
    all_segments = []
    
    #for text in texts:
    for i in tqdm(range(len(test_df))):
        text = tokenizer.tokenize(test_df.iloc[i])
        
        text = text[:max_len-2]
        input_sequence = ["[CLS]"] + text + ["[SEP]"]
        pad_len = max_len - len(input_sequence)
        
        tokens = tokenizer.convert_tokens_to_ids(input_sequence)
        tokens += [0] * pad_len
        pad_masks = [1] * len(input_sequence) + [0] * pad_len
        segment_ids = [0] * max_len
        
        all_tokens.append(tokens)
        all_masks.append(pad_masks)
        all_segments.append(segment_ids)
    
    return np.array(all_tokens), np.array(all_masks), np.array(all_segments)

def build_model(bert_layer, max_len=512):
    input_word_ids = Input(shape=(max_len,), dtype=tf.int32, name="input_word_ids")
    input_mask = Input(shape=(max_len,), dtype=tf.int32, name="input_mask")
    segment_ids = Input(shape=(max_len,), dtype=tf.int32, name="segment_ids")

    _, sequence_output = bert_layer([input_word_ids, input_mask, segment_ids])
    clf_output = sequence_output[:, 0, :]
    print(clf_output.shape)
    #model ok #f1 = 82 % 
    flat_layer = Flatten()(clf_output)
    dropout= Dropout(0.2)(flat_layer)

    out = Dense(10, activation='softmax')(dropout)
    #todo: provare a mettere una cnn al posto del semplice dense
    #c'è un po di overfitting
   
    model = Model(inputs=[input_word_ids, input_mask, segment_ids], outputs=out)
    model.compile(Adam(lr=2e-6), loss='categorical_crossentropy', metrics=['accuracy', get_f1])

    return model



In [39]:
def training_with_bert(df, X, y, modelName):
  
  #split dataset
  X_train, X_test, y_train, y_test = model_selection.train_test_split(X, y, test_size=0.3, random_state = 42)
  #X_train, X_test, y_train, y_test, X_val, y_val = split_dataset(df.iloc[:, 0 ],df.iloc[:, 3])
  y_train = to_categorical(np.asarray(y_train)) #y 
  y_test = to_categorical(np.asarray(y_test)) #y
  #y_val = to_categorical(np.asarray(y_val))

  module_url = "https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/1"
  #module_url = "https://tfhub.dev/tensorflow/bert_en_uncased_L-24_H-1024_A-16/1"

  bert_layer = hub.KerasLayer(module_url, trainable=True)

  vocab_file = bert_layer.resolved_object.vocab_file.asset_path.numpy()
  do_lower_case = bert_layer.resolved_object.do_lower_case.numpy()

  tokenizer = tokenization.FullTokenizer(vocab_file, do_lower_case)
  train_input = bert_encode(X_train, tokenizer, max_len=128)
  test_input = bert_encode(X_test, tokenizer, max_len=128)
  #val_input = bert_encode(X_val, tokenizer,  max_len=128)

  if not os.path.isdir(modelName) :
    print("models are training...")

    bert = build_model(bert_layer, max_len=128)
    bert.summary()
    es_callback = EarlyStopping(monitor='val_loss', patience=3)
    history = bert.fit(
          train_input, y_train,
          validation_split=0.2, 
          epochs=25,
          batch_size=16,
          callbacks=[es_callback]
    )
    bert.save(modelName)
    print("history", history)
    hist_df = pd.DataFrame(history.history) 

    hist_csv_file = './drive/MyDrive/history/history' + modelName.replace("./drive/MyDrive/", "") + '.csv'   
    with open(hist_csv_file, mode='w') as f:
      hist_df.to_csv(f)

    plot_accuracy_loss(history)
    getPerformanceMetricsBERT(bert, test_input, y_test)
  else: 
    #load trained models
    print("models already trained, loading...")
    bert = tf.keras.models.load_model(modelName,  custom_objects={'get_f1': get_f1})
   # bert_no_preprocess = tf.keras.models.load_model('./drive/MyDrive/BERT_model_preProcessing',  custom_objects={'get_f1': get_f1}) 
  
  return bert, test_input, y_test


In [ ]:
df = get_data()
df , categories= pre_processing(df)

categoryDistribution = df.groupby(["label"])["content"].count()
plt.pyplot.bar(categories, categoryDistribution, width = 0.7, bottom=None, align='center', data=None)
#visualize mean of sentence lenght for each category
df.insert(1, 'number_of_words', df.cleaned_text.apply(lambda x: len(x)))
median_words_for_sentence = df.groupby("label")["number_of_words"].median()
plt.pyplot.bar(categories, median_words_for_sentence, width = 0.8, bottom=None, align='center', data=None)

#df.groupby('label').apply(lambda x: word_cloud(x))

#cast cleaned text into string
df['cleaned_text'] = df['cleaned_text'].apply(lambda x: ', '.join(map(str, x)))
df['cleaned_text'] = df['cleaned_text'].apply(lambda x: x.replace(",", ""))
#X_train, X_test, y_train, y_test, X_val, y_val = split_dataset(df.iloc[:, 0 ],df.iloc[:, 4])  #cleaned text and string label
compute_baseline(df)
"""
LSTM_model_custom_we, CNN_model_custom_we, x_test, y_test = training_over_custom_word_embedding(df)

getPerformanceMetricsBERT(LSTM_model_custom_we, x_test, y_test, "LSTM_model_custom_we_skipgram" )
getPerformanceMetricsBERT(CNN_model_custom_we, x_test, y_test, "CNN_model_custom_we_skipgram" )


LSTM_model_glove_we, CNN_model_glove_we, x_test, y_test = training_over_glove_word_embedding(df)
getPerformanceMetricsBERT(CNN_model_glove_we, x_test, y_test, "CNN_model_glove_we" )
getPerformanceMetricsBERT(LSTM_model_glove_we, x_test, y_test , "LSTM_model_glove_we")
"""
cleaned_text = df["cleaned_text"]
raw_text = df["content"]
bert_no_preprocess, x_test, y_test = training_with_bert(df, raw_text, df.iloc[:, 4], "./drive/MyDrive/BERT_model_no_preProcessing1")
getPerformanceMetricsBERT(bert_no_preprocess, x_test, y_test, "bert_no_preprocess" )

bert_preprocess, x_test, y_test = training_with_bert(df, cleaned_text, df.iloc[:, 4], "./drive/MyDrive/BERT_model_preProcessing1")
getPerformanceMetricsBERT(bert_preprocess, x_test, y_test, "bert_preprocess" )


Dataset has already been downloaded. 
Dataset files have already been joined. 


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  from ipykernel import kernelapp as app
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


accuracy 0.7420596727622714
              precision    recall  f1-score   support

      Report       0.81      0.59      0.68        73
        Note       0.86      0.97      0.91       184
       Email       0.78      0.83      0.81       132
        ADVE       0.74      0.75      0.74       169
      Letter       0.68      0.85      0.75       183
  Scientific       0.58      0.84      0.68        49
      Resume       0.73      0.49      0.59        65
        News       0.78      0.19      0.31        93
        Memo       0.90      1.00      0.95        27
        Form       0.60      0.62      0.61        64

    accuracy                           0.74      1039
   macro avg       0.74      0.71      0.70      1039
weighted avg       0.75      0.74      0.72      1039



100%|██████████| 1039/1039 [00:04<00:00, 238.10it/s]


models are training...
(None, 768)
Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_word_ids (InputLayer)     [(None, 128)]        0                                            
__________________________________________________________________________________________________
input_mask (InputLayer)         [(None, 128)]        0                                            
__________________________________________________________________________________________________
segment_ids (InputLayer)        [(None, 128)]        0                                            
__________________________________________________________________________________________________
keras_layer (KerasLayer)        [(None, 768), (None, 109482241   input_word_ids[0][0]             
                                                           

In [ ]:
#to do unica funzione per ogni possibile modello, se già fatto il training e salvato, allora non fare più il training ma salvare 
#per ogni modello dare le metriche, dopo di chè magari provare a fare la classificazione di un singolo sample.


In [ ]:
#single plot for f1 values, and loss values got from each model 
def compare_curves(): 
  plt.pyplot.subplot(211)
  plt.pyplot.title('model f1_score')
  plt.pyplot.ylabel('f1_score')
  plt.pyplot.xlabel('epoch')
  mod = []
  
  for dirpath, dirs, filenames in os.walk("./drive/MyDrive/history/"):
    for filename in filenames:
      mod.append(filename)
      hist_df = pd.read_csv(dirpath +"/"+filename, sep=',', delimiter=None )
      plt.pyplot.plot(hist_df['val_get_f1'])
  
  plt.pyplot.subplot(212)
  plt.pyplot.title('model loss value')
  plt.pyplot.ylabel('loss')
  plt.pyplot.xlabel('epoch')

  for dirpath, dirs, filenames in os.walk("./drive/MyDrive/history/"):
    for filename in filenames:
      mod.append(filename)
      hist_df = pd.read_csv(dirpath +"/"+filename, sep=',', delimiter=None )
      plt.pyplot.plot(hist_df['val_loss'])
  


  plt.pyplot.legend(mod, loc='upper left')
  plt.pyplot.show()

  
compare_curves()

In [ ]:
hist_df = pd.read_csv("./drive/MyDrive/history/historyCNN_model_custom_WE1.csv", sep=',', delimiter=None )  
hist_df